In [2]:
from itertools import permutations, product
import numpy as np
from tqdm.autonotebook import tqdm, trange

def rotmat():
  for x, y, z in permutations([0, 1, 2]):
    for sx, sy, sz in product([-1, 1], repeat=3):
      R = np.zeros((3, 3))
      R[0, x] = sx
      R[1, y] = sy
      R[2, z] = sz
      if np.linalg.det(R) == 1: yield R

def read(path):
  scanners = []
  with open(path) as inf:
    scanner = []
    for line in inf:
      line = line.strip()
      if not line: continue
      if '--- scanner' in line:
        if scanner: scanners.append(np.array(scanner))
        scanner = []
      else:
        scanner.append(eval(line))
  if scanner: scanners.append(np.array(scanner))      
  return scanners

Rs = list(rotmat())

In [3]:
SCANNERS = read('input0.txt')
len(SCANNERS)

5

In [233]:
s0 = np.array([
  [-618,-824,-621],
  [-537,-823,-458],
  [-447,-329,318],
  [404,-588,-901],
  [544,-627,-890],
  [528,-643,409],
  [-661,-816,-575],
  [390,-675,-793],
  [423,-701,434],
  [-345,-311,381],
  [459,-707,401],
  [-485,-357,347]
])
s1 = np.array([
  [686,422,578],
  [605,423,415],
  [515,917,-361],
  [-336,658,858],
  [-476,619,847],
  [-460,603,-452],
  [729,430,532],
  [-322,571,750],
  [-355,545,-477],
  [413,935,-424],
  [-391,539,-444],
  [553,889,-390]
])
output = frozenset([
  (-892,524,684),
  (-876,649,763),
  (-838,591,734),
  (-789,900,-551),
  (-739,-1745,668),
  (-706,-3180,-659),
  (-697,-3072,-689),
  (-689,845,-530),
  (-687,-1600,576),
  (-661,-816,-575),
  (-654,-3158,-753),
  (-635,-1737,486),
  (-631,-672,1502),
  (-624,-1620,1868),
  (-620,-3212,371),
  (-618,-824,-621),
  (-612,-1695,1788),
  (-601,-1648,-643),
  (-584,868,-557),
  (-537,-823,-458),
  (-532,-1715,1894),
  (-518,-1681,-600),
  (-499,-1607,-770),
  (-485,-357,347),
  (-470,-3283,303),
  (-456,-621,1527),
  (-447,-329,318),
  (-430,-3130,366),
  (-413,-627,1469),
  (-345,-311,381),
  (-36,-1284,1171),
  (-27,-1108,-65),
  (7,-33,-71),
  (12,-2351,-103),
  (26,-1119,1091),
  (346,-2985,342),
  (366,-3059,397),
  (377,-2827,367),
  (390,-675,-793),
  (396,-1931,-563),
  (404,-588,-901),
  (408,-1815,803),
  (423,-701,434),
  (432,-2009,850),
  (443,580,662),
  (455,729,728),
  (456,-540,1869),
  (459,-707,401),
  (465,-695,1988),
  (474,580,667),
  (496,-1584,1900),
  (497,-1838,-617),
  (527,-524,1933),
  (528,-643,409),
  (534,-1912,768),
  (544,-627,-890),
  (553,345,-567),
  (564,392,-477),
  (568,-2007,-577),
  (605,-1665,1952),
  (612,-1593,1893),
  (630,319,-379),
  (686,-3108,-505),
  (776,-3184,-501),
  (846,-3110,-434),
  (1135,-1161,1235),
  (1243,-1093,1063),
  (1660,-552,429),
  (1693,-557,386),
  (1735,-437,1738),
  (1749,-1800,1813),
  (1772,-405,1572),
  (1776,-675,371),
  (1779,-442,1789),
  (1780,-1548,337),
  (1786,-1538,337),
  (1847,-1591,415),
  (1889,-1729,1762),
  (1994,-1805,1792)
])
len(output)

79

In [21]:
def as_set(arr):
  return set(tuple(r) for r in arr)

def tra_rot(t, s):
  for s_row in s:
    st = as_set(s - s_row)
    for m in Rs:
      r = (t @ m).astype(int)
      for r_row in r:
        i = st & as_set(r - r_row)
        if len(i) >= 12: return s_row - r_row, m
  return None, None

def transf(frm, tra, rot):
  return (tra + frm @ rot).astype(int)

def intersect(s, t):
  return np.array(list(as_set(s) & as_set(t)))

def union(s, t):
  return np.array(list(as_set(s) | as_set(t)))

def invert(tra, rot):
  rot = np.linalg.inv(rot)
  tra = - tra.T @ rot 
  return tra, rot

In [24]:
s, t = SCANNERS[1], SCANNERS[0]
tra_st, rot_st = tra_rot(s, t)
i_t = intersect(transf(s, tra_st, rot_st), t)
i_t

array([[ 390, -675, -793],
       [-345, -311,  381],
       [ 404, -588, -901],
       [ 459, -707,  401],
       [-661, -816, -575],
       [ 544, -627, -890],
       [ 528, -643,  409],
       [-485, -357,  347],
       [-537, -823, -458],
       [ 423, -701,  434],
       [-618, -824, -621],
       [-447, -329,  318]])

In [7]:
tra_ts, rot_ts = invert(tra_st, rot_st)
i_s = intersect(transf(t, tra_ts, rot_ts), s)
i_s

array([[-476,  619,  847],
       [-336,  658,  858],
       [-391,  539, -444],
       [ 729,  430,  532],
       [-322,  571,  750],
       [-355,  545, -477],
       [ 605,  423,  415],
       [ 515,  917, -361],
       [ 553,  889, -390],
       [ 686,  422,  578],
       [ 413,  935, -424],
       [-460,  603, -452]])

In [8]:
as_set(transf(i_t, tra_ts, rot_ts)) == as_set(i_s)

True

In [36]:
def compose(t, s, tra_ts, rot_ts, bt):
  def _bt(i):
    print(f'{t} -> {s}')
    return bt(transf(i, tra_ts, rot_ts))
  return _bt

def tid(i):
  print('id')
  return i

def visit(s, bt):
  for t in set(range(len(SCANNERS))) - seen:
    tra_ts, rot_ts = tra_rot(SCANNERS[t], SCANNERS[s])
    if tra_ts is None: continue
    i = bt(transf(SCANNERS[t], tra_ts, rot_ts))
    print(f'{t} -> {s}')
    print(i)
    found.append(i)
    seen.add(t)
    visit(t, compose(t, s, tra_ts, rot_ts, bt))

seen = set([0])
found = []
visit(0, tid)
print(found)

id
1 -> 0
[[ 390 -675 -793]
 [-345 -311  381]
 [ 404 -588 -901]
 [ 459 -707  401]
 [-661 -816 -575]
 [ 544 -627 -890]
 [ 528 -643  409]
 [-485 -357  347]
 [-537 -823 -458]
 [ 423 -701  434]
 [-618 -824 -621]
 [-447 -329  318]]
1 -> 0
id
3 -> 1
[[  534 -1912   768]
 [ -601 -1648  -643]
 [ -635 -1737   486]
 [  432 -2009   850]
 [  408 -1815   803]
 [ -499 -1607  -770]
 [  568 -2007  -577]
 [  497 -1838  -617]
 [ -687 -1600   576]
 [ -739 -1745   668]
 [  396 -1931  -563]
 [ -518 -1681  -600]]
1 -> 0
id
4 -> 1
[[  534 -1912   768]
 [  459  -707   401]
 [ -635 -1737   486]
 [  423  -701   434]
 [  432 -2009   850]
 [  408 -1815   803]
 [ -447  -329   318]
 [ -485  -357   347]
 [ -687 -1600   576]
 [ -739 -1745   668]
 [ -345  -311   381]
 [  528  -643   409]]
4 -> 1
1 -> 0
id
2 -> 4
[[  459  -707   401]
 [  534 -1912   768]
 [  408 -1815   803]
 [  432 -2009   850]
 [  456  -540  1869]
 [  423  -701   434]
 [  496 -1584  1900]
 [  612 -1593  1893]
 [  527  -524  1933]
 [  528  -643   409]

In [34]:
res = set()
for bs in found:
  res |= as_set(bs)
len(res)

30